In [33]:
import pandas as pd
import os
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [34]:
#Read review data from csv
reviews = pd.read_csv("C:/Users/wgvan/Desktop/reviews.csv", sep=',', engine = 'python', quotechar='\"', escapechar='\\')
#drop irrelevant review columns
reviews = reviews.drop(columns=['id', 'workId', 'reviewId', 'postTime', 'episodesSeen', 'author', 'peopleFoundUseful'], axis=1)
reviews.head()

,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...
1,Utawarerumono,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...
2,Hajime_no_Ippo,10,10,9,9,10,10,"First, let me say that I\'m not a fan of boxin..."
3,Gensoumaden_Saiyuuki,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...
4,Ranma_Â½,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...


In [35]:
#Read genre data from csv
genres = pd.read_csv("C:/Users/wgvan/Desktop/anime.csv", sep=',', engine = 'python', quotechar='\"', escapechar='\\')
genres.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,GintamaÂ°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [36]:
#replace some punctuation and spaces with _ so that the names in the two files are the same
genres['name'].replace('\s+', '_',regex=True,inplace=True)
genres['name'].replace(':', '_',regex=True,inplace=True)
genres['name'].replace(';', '_',regex=True,inplace=True)
genres['name'].replace('&', '_',regex=True,inplace=True)
genres['name'].replace('\'', '',regex=True,inplace=True)
genres['name'].replace('!', '',regex=True,inplace=True)
genres.head()

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<ipython-input-36-92fda4bd457d>:2: DeprecationWarning: invalid escape sequence \s
  genres['name'].replace('\s+', '_',regex=True,inplace=True)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi_no_Na_wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal_Alchemist__Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,GintamaÂ°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins_Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama_#039_,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [37]:
#make a dictionary out of the two needed genre columns
genre_dict = pd.Series(genres.genre.values, index = genres.name).to_dict()
#add genre column to the reviews
reviews['genre'] = reviews['workName'].map(genre_dict)
reviews['genre'].replace(',', '',regex=True,inplace=True)
reviews.head()

,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review,genre
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...,Action Adventure Comedy Drama Sci-Fi Space
1,Utawarerumono,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...,Action Drama Fantasy Sci-Fi
2,Hajime_no_Ippo,10,10,9,9,10,10,"First, let me say that I\'m not a fan of boxin...",Comedy Drama Shounen Sports
3,Gensoumaden_Saiyuuki,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...,Adventure Comedy Drama Shounen Supernatural
4,Ranma_Â½,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...,Comedy Fantasy Martial Arts Slice of Life


In [38]:
action = reviews[reviews['genre'].str.contains('Action', na = False)]
print(len(action.index))
action.head()

41955


,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review,genre
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...,Action Adventure Comedy Drama Sci-Fi Space
1,Utawarerumono,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...,Action Drama Fantasy Sci-Fi
5,Kino_no_Tabi__The_Beautiful_World,9,9,9,8,9,9,What do you say about an anime that is just ab...,Action Adventure Slice of Life
7,Hunter_x_Hunter,10,10,8,9,10,10,Overall: My very best anime and i haven't act...,Action Adventure Shounen Super Power
9,Hunter_x_Hunter,10,10,8,8,10,10,Huntersâ€¦ What do you think about when you he...,Action Adventure Shounen Super Power


In [39]:
#remove punctiation and convert everything to lowercase
action['review_processed'] = action['review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
action['review_processed'] = action['review_processed'].map(lambda x: x.lower())
action['review_processed'].head()

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
<ipython-input-39-e360ea516ad8>:2: DeprecationWarning: invalid escape sequence \.
  action['review_processed'] = action['review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
<ipython-input-39-e360ea516ad8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  action['review_processed'] = action['review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
<ipython-input-39-e360ea516ad8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

0    cowboy bebop is an episodic series by episodic...
1    utawarerumono manages to be one of those harem...
5    what do you say about an anime that is just ab...
7    overall:  my very best anime and i haven't act...
9    huntersâ€¦ what do you think about when you he...
Name: review_processed, dtype: object

In [40]:
def toWords(sentences):
    for s in sentences:
        yield(gensim.utils.simple_preprocess(str(s), deacc=True))
        
data = action.review_processed.values.tolist()
data_words = list(toWords(data))

print(data_words[:1])

[['cowboy', 'bebop', 'is', 'an', 'episodic', 'series', 'by', 'episodic', 'mean', 'that', 'one', 'episode', 'doesn', 'necessarily', 'lead', 'or', 'follow', 'the', 'next', 'one', 'with', 'that', 'being', 'said', 'don', 'expect', 'lot', 'of', 'plot', 'in', 'this', 'series', 'you', 'watch', 'cowboy', 'bebop', 'for', 'the', 'characters', 'plain', 'and', 'simple', 'spike', 'faye', 'jet', 'ed', 'ein', 'are', 'what', 'make', 'this', 'anime', 'arguably', 'one', 'of', 'the', 'most', 'memorable', 'series', 'ever', 'produced', 'the', 'music', 'is', 'awesome', 'especially', 'if', 'you', 're', 'fan', 'of', 'jazz', 'the', 'animation', 'is', 'top', 'notch', 'considering', 'when', 'the', 'series', 'came', 'out', 'and', 'the', 'enjoyment', 'value', 'is', 'second', 'to', 'nonewhile', 'lot', 'of', 'people', 'want', 'that', 'similar', 'episode', 'feel', 'if', 'you', 're', 'the', 'kind', 'of', 'person', 'that', 'enjoys', 'fast', 'paced', 'anime', 'with', 'each', 'character', 'getting', 'his', 'or', 'her', '

In [41]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [42]:
#remove stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [43]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model
nlp = en_core_web_sm.load()
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

C:\Users\wgvan\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.3). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[['mean', 'episode', 'necessarily', 'lead', 'follow', 'next', 'say', 'expect', 'lot', 'plot', 'character', 'plain', 'simple', 'spike', 'make', 'arguably', 'memorable', 'series', 'ever', 'produce', 'music', 'awesome', 'especially', 'fan', 'jazz', 'animation', 'consider', 'come', 'enjoyment', 'value', 'second', 'nonewhile', 'lot', 'people', 'want', 'similar', 'episode', 'feel', 'kind', 'person', 'enjoy', 'fast_pace', 'anime', 'character', 'get', 'spotlight', 'type', 'enjoy', 'fast_pace', 'fun', 'rather', 'deep', 'thought_provoking', 'dark', 'mysterious', 'watch', 'cowboy_bebop', 'vicious', 'character', 'able', 'bring', 'little', 'sinister', 'side', 'kind', 'show']]


In [44]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)]]


In [45]:
# Build LDA model (alpha, eta default, num_topics = 10)
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

# Compute Coherence Score and perplexity
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Coherence Score:  0.3303526104595812

Perplexity:  -7.8863616558614495


In [46]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.183303  0.028094       1        1  39.649059
0     -0.151874  0.047034       2        1  20.854738
9     -0.160161 -0.020122       3        1  16.098639
4     -0.128669 -0.064884       4        1   8.481186
1     -0.109542 -0.050242       5        1   5.530259
5     -0.092238  0.039225       6        1   4.260944
8     -0.044842 -0.201149       7        1   3.000600
2      0.000586  0.245758       8        1   1.823480
3      0.441176 -0.012861       9        1   0.192830
7      0.428865 -0.010852      10        1   0.108263, topic_info=             Term          Freq         Total Category  logprob  loglift
1271       season  29270.000000  29270.000000  Default  30.0000  30.0000
110         first  46450.000000  46450.000000  Default  29.0000  29.0000
277         movie  20019.000000  20019.000000  Default  28.0000  28.0000
559          girl  18042.000000  18042.000000  Default  27.0000  27.0000
44         second  15837.000000  15837.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
3156   generation     42.858860    672.520447  Topic10  -5.3417   4.0752
10685        dawn     12.860006    145.944534  Topic10  -6.5455   4.3993
118          game     18.406258  12723.715820  Topic10  -6.1869   0.2898
6818    nostalgia     13.581979    523.271729  Topic10  -6.4909   3.1770
5676    childhood     13.035369    835.076721  Topic10  -6.5320   2.6685

[766 rows x 6 columns], token_table=       Topic      Freq               Term
term                                     
21764      8  0.997313           abnormal
39498      9  0.957559            abyssal
46160      1  0.993533        accel_world
59         1  0.434093             action
59         2  0.373155             action
...      ...       ...                ...
42706      1  0.997276               yuno
20062      8  0.996364              yuuki
5443       4  0.002267             zombie
5443       7  0.997468             zombie
32309      7  0.996756  zombie_apocalypse

[2237 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 10, 5, 2, 6, 9, 3, 4, 8])